In [33]:
import dash
from dash import dcc, html, Input, Output
import dash_bootstrap_components as dbc
import plotly.graph_objects as go
import numpy as np
import pandas as pd
import dash_ag_grid as dag


num_rows = 10000

# Создаем примеры данных
data = {
    'Дата': pd.date_range(start='2023-01-01', periods=num_rows, freq='h'),
    'Контрагент': np.random.choice(['Компания А', 'Компания B', 'Компания C', 'Компания D', 
                                    'Компания E', 'Компания F', 'Компания G', 'Компания H', 
                                    'Компания I', 'Компания J'], size=num_rows),
    'Сумма': np.random.randint(-1000, 50000, size=num_rows),
    'Договор': [f'Договор №{i}' for i in np.random.randint(1, 1000, size=num_rows)],
    'Статья учета': np.random.choice(['Услуги', 'Товары', 'Аренда', 'Зарплата', 'Командировки', 
                                      'Закупки', 'Прочее', 'Капитальные затраты', 'Реклама', 
                                      'Консультации'], size=num_rows),
    'Банковский счет': [f'Счет №{np.random.randint(1000, 9999)}' for _ in range(num_rows)]
}

df_filtered = pd.DataFrame(data)

dataTypeDefinitions = {
    "object": {
        "baseDataType": "object",
        "extendsDataType": "object",
        "valueParser": {"function": "({ name: params.newValue })"},
        "valueFormatter": {"function": "params.value == null ? '' : params.value.name"},
    },
}

col_defs = []
for col in df.columns:
    if pd.api.types.is_numeric_dtype(df[col]):
        col_type = 'Number'
    elif pd.api.types.is_datetime64_any_dtype(df[col]):
        col_type = 'dateString'
    else:
        col_type = 'text'
    col_defs.append({"field": col, 'editable': True})

datatable = dag.AgGrid(id='datatable',
                       rowData=df.to_dict('records'),
                       columnDefs=col_defs,
                       dashGridOptions={"dataTypeDefinitions": dataTypeDefinitions, "animateRows": False},
                       suppressDragLeaveHidesColumns=False,
                       persistence=True,
                       defaultColDef={'filter': True,
                                      'floatingFilter': True,
                                      'resizable': True,
                                      'sortable': True,
                                      'editable': False,
                                      'minWidth': 125,
                                      'wrapHeaderText': True,
                                      'autoHeaderHeight': True},
                       style={'resize': 'both', 'overflow': 'hidden'}
                       )
fig_contract = go.Figure()
fig_contract.add_trace(
    go.Bar(
            x=df['Статья учета'], 
            y=df['Сумма'], 
            name='Статья'))
fig_contract.add_trace(go.Bar(
        x=df['Контрагент'], 
        y=df['Сумма'], 
        name='Контрагент'))
fig_contract.update_layout(
        title=' ', 
        barmode='group')
    

# Фильтрация данных по расходам и доходам
expenses_df = df_filtered[df_filtered['Сумма'] < 0].copy()
incomes_df = df_filtered[df_filtered['Сумма'] > 0].copy()

# Преобразование отрицательных сумм расходов в положительные для корректного отображения в treemap
expenses_df['Сумма'] = expenses_df['Сумма'].abs()

# Группировка по 'Статья учета' и суммирование по 'Сумма' для расходов
grouped_expenses = expenses_df.groupby('Статья учета')['Сумма'].sum().reset_index()
grouped_expenses_ = expenses_df.groupby(['Статья учета', 'Контрагент'])['Сумма'].sum().reset_index()

# Генерация уникальных идентификаторов
expense_account_ids = [f"exp_{i}" for i in range(len(grouped_expenses))]
expense_contractor_ids = [f"exp_{i}_{j}" for i in range(len(grouped_expenses)) for j in range(len(grouped_expenses_.loc[grouped_expenses_['Статья учета'] == grouped_expenses.iloc[i, 0]]))]

# Создание списков для treemap по расходам
expenses_labels = list(grouped_expenses['Статья учета']) + list(grouped_expenses_['Контрагент'])
expenses_parents = [''] * len(grouped_expenses['Статья учета']) + [f"exp_{i}" for i in range(len(grouped_expenses)) for _ in range(len(grouped_expenses_.loc[grouped_expenses_['Статья учета'] == grouped_expenses.iloc[i, 0]]))]
expenses_values = list(grouped_expenses['Сумма']) + list(grouped_expenses_['Сумма'])
expenses_ids = expense_account_ids + expense_contractor_ids

# Создание графика treemap для расходов
fig_expenses = go.Figure(go.Treemap(
    ids=expenses_ids,
    labels=expenses_labels,
    parents=expenses_parents,
    values=expenses_values
))
fig_expenses.update_layout(title="Расходы")

# Группировка по 'Статья учета' и суммирование по 'Сумма' для доходов
grouped_incomes = incomes_df.groupby('Статья учета')['Сумма'].sum().reset_index()
grouped_incomes_ = incomes_df.groupby(['Статья учета', 'Контрагент'])['Сумма'].sum().reset_index()

# Генерация уникальных идентификаторов
income_account_ids = [f"inc_{i}" for i in range(len(grouped_incomes))]
income_contractor_ids = [f"inc_{i}_{j}" for i in range(len(grouped_incomes)) for j in range(len(grouped_incomes_.loc[grouped_incomes_['Статья учета'] == grouped_incomes.iloc[i, 0]]))]

# Создание списков для treemap по доходам
incomes_labels = list(grouped_incomes['Статья учета']) + list(grouped_incomes_['Контрагент'])
incomes_parents = [''] * len(grouped_incomes['Статья учета']) + [f"inc_{i}" for i in range(len(grouped_incomes)) for _ in range(len(grouped_incomes_.loc[grouped_incomes_['Статья учета'] == grouped_incomes.iloc[i, 0]]))]
incomes_values = list(grouped_incomes['Сумма']) + list(grouped_incomes_['Сумма'])
incomes_ids = income_account_ids + income_contractor_ids

# Создание графика treemap для доходов
fig_incomes = go.Figure(go.Treemap(
    ids=incomes_ids,
    labels=incomes_labels,
    parents=incomes_parents,
    values=incomes_values
))
fig_incomes.update_layout(title="Доходы")

print(expenses_ids)
print(expenses_labels)
print(expenses_parents)  

['exp_0', 'exp_1', 'exp_2', 'exp_3', 'exp_4', 'exp_5', 'exp_6', 'exp_7', 'exp_8', 'exp_9', 'exp_0_0', 'exp_0_1', 'exp_0_2', 'exp_0_3', 'exp_0_4', 'exp_0_5', 'exp_0_6', 'exp_0_7', 'exp_0_8', 'exp_1_0', 'exp_1_1', 'exp_1_2', 'exp_1_3', 'exp_1_4', 'exp_1_5', 'exp_1_6', 'exp_1_7', 'exp_2_0', 'exp_2_1', 'exp_2_2', 'exp_2_3', 'exp_2_4', 'exp_2_5', 'exp_2_6', 'exp_2_7', 'exp_2_8', 'exp_3_0', 'exp_3_1', 'exp_3_2', 'exp_3_3', 'exp_3_4', 'exp_3_5', 'exp_3_6', 'exp_3_7', 'exp_3_8', 'exp_4_0', 'exp_4_1', 'exp_4_2', 'exp_4_3', 'exp_4_4', 'exp_4_5', 'exp_4_6', 'exp_4_7', 'exp_4_8', 'exp_4_9', 'exp_5_0', 'exp_5_1', 'exp_5_2', 'exp_5_3', 'exp_5_4', 'exp_5_5', 'exp_5_6', 'exp_5_7', 'exp_5_8', 'exp_6_0', 'exp_6_1', 'exp_6_2', 'exp_6_3', 'exp_6_4', 'exp_6_5', 'exp_6_6', 'exp_6_7', 'exp_6_8', 'exp_6_9', 'exp_7_0', 'exp_7_1', 'exp_7_2', 'exp_7_3', 'exp_7_4', 'exp_8_0', 'exp_8_1', 'exp_8_2', 'exp_8_3', 'exp_8_4', 'exp_8_5', 'exp_8_6', 'exp_8_7', 'exp_9_0', 'exp_9_1', 'exp_9_2', 'exp_9_3', 'exp_9_4', 'exp_9_